In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/mnist_improved_tdt4265_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-03-23 15:00:03,107 SSD INFO: Loaded configuration file configs/mnist_tdt4265_server.yaml
2021-03-23 15:00:03,108 SSD INFO: 
MODEL:
    NUM_CLASSES: 11
    BACKBONE:
        NAME: 'improved'
        PRETRAINED: False
        OUT_CHANNELS: [512, 1024, 512, 512, 256, 256]
        INPUT_CHANNELS: 3
INPUT:
    IMAGE_SIZE: [300, 300]
DATASETS:
    TRAIN: ("mnist_detection_train", "mnist_detection_val")
    TEST: ("mnist_detection_val", )
SOLVER:
    MAX_ITER: 10000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 2e-3
OUTPUT_DIR: 'outputs/basic'
DATASET_DIR: "/work/datasets"

2021-03-23 15:00:03,110 SSD INFO: Running with config:
DATASETS:
  TEST: ('mnist_detection_val',)
  TRAIN: ('mnist_detection_train', 'mnist_detection_val')
DATASET_DIR: /work/datasets
DATA_LOADER:
  NUM_WORKERS: 4
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: [300, 300]
  PIXEL_MEAN: [123.675, 116.28, 103.53]
  PIXEL_STD: [1, 1, 1]
LOG_STEP: 10
MODEL:
  BACKBONE:
    INPUT_CHANNELS: 3
    NAME: improved
    OUT_C

In [4]:
model = start_train(cfg)

Detector initialized. Total Number of params:  36.15M
Backbone number of parameters: 33.94M
SSD Head number of parameters: 2.21M
2021-03-23 15:00:10,060 SSD.trainer INFO: Loading checkpoint from outputs/basic/model_final.pth
2021-03-23 15:00:10,602 SSD.trainer INFO: Loading optimizer from outputs/basic/model_final.pth
2021-03-23 15:00:12,073 SSD.trainer INFO: Start training ...
2021-03-23 15:00:15,496 SSD.trainer INFO: Saving checkpoint to outputs/basic/model_final.pth
2021-03-23 15:00:16,677 SSD.trainer INFO: Total training time: 0:00:03 (0.0003 s / it)


In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-03-23 15:00:19,172 SSD INFO: Start evaluating...
2021-03-23 15:00:19,326 SSD.inference INFO: Evaluating mnist_detection_val dataset(1000 images):


100%|██████████| 100/100 [00:12<00:00,  7.69it/s]


2021-03-23 15:00:33,086 SSD.inference INFO: mAP: 0.8753
0               : 0.8940
1               : 0.8095
2               : 0.8760
3               : 0.8971
4               : 0.8807
5               : 0.8830
6               : 0.8859
7               : 0.8569
8               : 0.8900
9               : 0.8794



[{'metrics': {'mAP': 0.8752565431542013,
   '0': 0.8939699650469805,
   '1': 0.8094709526574602,
   '2': 0.8760044641851805,
   '3': 0.897146832405082,
   '4': 0.8807326189016644,
   '5': 0.883031870897962,
   '6': 0.8859388314999235,
   '7': 0.8569403048356649,
   '8': 0.8899598842157962,
   '9': 0.8793697068962991}}]